Chắc chắn rồi Vinh! Dưới đây là phần **giải thích lại từ đầu** theo đúng 3 ý: **Vấn đề → Ý tưởng → Triển khai**, súc tích nhưng dễ hình dung.

---

## 🎯 **1. Vấn đề cần giải quyết**

Bạn có **một chuỗi lớn `text`** và cần tìm nhanh một (hoặc nhiều) **mẫu con `pattern`** nằm trong đó.

Ví dụ:

* Tìm xem từ `"error"` có xuất hiện trong log dài 100MB không?
* Phát hiện câu bị đạo văn: `"the quick brown fox"` có nằm trong tài liệu nào không?
* Trong AI, cần biết liệu `"transformer"` có xuất hiện trong hàng triệu tiêu đề bài viết hay không?

👉 Nếu duyệt lần lượt từng ký tự để so sánh thì tốn **O(N·M)** (rất chậm khi `text` lớn).

---

## 💡 **2. Ý tưởng tiếp cận**

Có 2 nhánh chính:

### A. **Hash chuỗi con để so nhanh** (Rabin-Karp)

→ biến chuỗi thành số nguyên (hash) → so sánh số thay vì từng ký tự
→ di chuyển cửa sổ `window` dài `m`, cập nhật hash **trượt (rolling)** → O(1) mỗi bước

### B. **Tận dụng cấu trúc lặp tiền tố** (KMP – Knuth Morris Pratt)

→ phân tích `pattern` để hiểu: nếu sai ở vị trí i, thì quay lại đâu là hiệu quả nhất?
→ tiền xử lý tạo bảng `lps` (longest prefix-suffix) để tránh so lại từ đầu

---

## 🔧 **3. Triển khai cụ thể**

### 👉 Cách A: Rabin-Karp (Rolling Hash)

```python
def rk_search(text, pattern):
    B, MOD = 256, 1_000_000_007
    n, m = len(text), len(pattern)
    if m == 0: return 0

    power = pow(B, m-1, MOD)   # B^(m-1) để trừ chữ cái cũ
    hash_text = hash_pat = 0
    for i in range(m):
        hash_text = (hash_text * B + ord(text[i])) % MOD
        hash_pat = (hash_pat * B + ord(pattern[i])) % MOD

    for i in range(n - m + 1):
        if hash_text == hash_pat and text[i:i+m] == pattern:
            return i
        if i < n - m:
            hash_text = (hash_text - ord(text[i]) * power) % MOD
            hash_text = (hash_text * B + ord(text[i + m])) % MOD
    return -1
```

* `ord(ch)` biến ký tự thành số
* Hash trượt theo công thức `remove + shift + add`
* Khi hash khớp → so ký tự để đảm bảo (tránh collision)

---

### 👉 Cách B: KMP (Prefix Border)

#### Bước 1: Xây bảng `lps`

```python
def build_lps(pat):
    lps = [0]*len(pat)
    j = 0
    for i in range(1, len(pat)):
        while j > 0 and pat[i] != pat[j]:
            j = lps[j-1]  # thu ngắn border
        if pat[i] == pat[j]:
            j += 1
        lps[i] = j
    return lps
```

#### Bước 2: Dùng `lps` để khớp

```python
def kmp_search(text, pattern):
    lps = build_lps(pattern)
    i = j = 0
    while i < len(text):
        if text[i] == pattern[j]:
            i += 1
            j += 1
        elif j > 0:
            j = lps[j-1]  # nhảy về prefix trùng
        else:
            i += 1
        if j == len(pattern):
            return i - j
    return -1
```

* `j` là con trỏ trong `pattern`, không cần reset về 0 mỗi lần mismatch
* Rất hiệu quả với dữ liệu có tính lặp (như DNA, logs, văn bản kỹ thuật)

---

Nếu bạn cần mình so sánh khi nào nên dùng hash, khi nào dùng prefix (hoặc multiple pattern → dùng trie/Aho-Corasick), mình có thể tiếp tục nhé!
Bạn muốn áp dụng với bài cụ thể nào để mình hướng dẫn triển khai tay không?


In [5]:
def rk_search(text, pattern):
    B, MOD = 256, 1_000_000_007
    n, m = len(text), len(pattern)
    print(f"Text: {text}, Pattern: {pattern}")
    print(f"Text length: {n}, Pattern length: {m}")
    if m == 0: return 0

    power = pow(B, m-1, MOD)   # B^(m-1) để trừ chữ cái cũ
    print(f"Power for rolling hash: {power}")
    hash_text = hash_pat = 0
    for i in range(m):
        hash_text = (hash_text * B + ord(text[i])) % MOD
        hash_pat = (hash_pat * B + ord(pattern[i])) % MOD
    cnt = 0
    for i in range(n - m + 1):
        if hash_text == hash_pat and text[i:i+m] == pattern:
            cnt += 1
        if i < n - m:
            hash_text = (hash_text - ord(text[i]) * power) % MOD
            hash_text = (hash_text * B + ord(text[i + m])) % MOD
    return cnt


print(rk_search("ababcababcabc", "abc"))  # Output: 2

Text: ababcababcabc, Pattern: abc
Text length: 13, Pattern length: 3
Power for rolling hash: 65536
3


Rất tốt, mình sẽ **giải thích kỹ bản chất của việc tính hash cho chuỗi**, đặc biệt là theo cách mà **Rabin-Karp** sử dụng – đảm bảo bạn hiểu được tại sao nó hoạt động, chứ không chỉ nhớ công thức.

---

## ❓ **Vì sao phải "hash" chuỗi?**

👉 Để biến chuỗi (nhiều ký tự) thành **một số duy nhất**, dễ **so sánh nhanh**
➡ Thay vì so từng ký tự (chậm), ta chỉ cần so 2 số (nhanh)

Ví dụ:
So sánh `"abcd"` với `"abcf"`

* Thường phải so 4 ký tự
* Nhưng nếu cả 2 chuỗi có **hash khác nhau**, ta biết chắc chắn **không giống**
  → Tiết kiệm công sức

---

## 🧠 **Cách tính hash: biến chuỗi thành số**

Ta sẽ dùng hệ cơ số **B = 256** (vì có 256 ký tự ASCII).
Mỗi ký tự là một **chữ số**, và chuỗi là một **số nguyên ở hệ B**.

### 🔢 Ví dụ:

Chuỗi: `"abc"`
Ký tự → số:

* `'a'` = 97
* `'b'` = 98
* `'c'` = 99

Hash =

```
= 97 * B^2 + 98 * B^1 + 99 * B^0  
= 97 * 256^2 + 98 * 256 + 99
```

Gộp lại:

```python
hash = (((0 * 256 + 97) * 256 + 98) * 256 + 99)
```

### ⚠️ Tại sao làm thế?

* Nó giống như cách bạn đọc số **123 = 1×100 + 2×10 + 3×1**
* Nhưng ở đây, "100" là `256^2`, vì mỗi ký tự là một số trong hệ 256

---

## 🔁 **Hash trượt (rolling hash)**

Giả sử bạn đang trượt cửa sổ trong `"abcdef"`, mỗi lần trượt 1 ký tự:

```
Lượt 1: "abc"
Lượt 2: "bcd"
Lượt 3: "cde"
...
```

Thay vì tính lại hash từ đầu mỗi lần, ta làm 2 bước:

1. **Trừ ký tự cũ** ra khỏi đầu (`a`)
2. **Thêm ký tự mới** vào cuối (`d`)

### Ví dụ đơn giản:

Giả sử:

* `"abc"` có hash là `H1`
* Để ra `"bcd"`, ta tính:

```python
H2 = (H1 - ord('a') * 256^2) * 256 + ord('d')
```

---

## 💡 **Mod để chống tràn số**

Vì các số có thể rất lớn (`256^10` chẳng hạn), ta luôn `% MOD` để:

* Giữ số nhỏ
* Giảm xác suất trùng (collision)

```python
MOD = 1_000_000_007  # số nguyên tố lớn
```

---

## ✅ Tóm lại: công thức chung

```python
hash = 0
for ch in s:
    hash = (hash * B + ord(ch)) % MOD
```

Mỗi ký tự đẩy hash lên một bậc và cộng giá trị mới vào.

---

## 📌 Mẹo ghi nhớ

* Hash = “cộng dồn từng ký tự” như cách đọc số trong hệ 256
* Rolling hash = “trượt cửa sổ, trừ đầu, thêm cuối”
* Luôn `% MOD` để chống tràn

---

Nếu bạn muốn, mình có thể viết đoạn code minh họa trực quan từng bước hash `"abc"` bằng `print()` để bạn thấy giá trị biến đổi sau mỗi ký tự. Bạn có muốn không?


In [27]:
def build_lps(pat):
    lps = [0]*len(pat)
    print(f"Building LPS for pattern: {pat}")
    j = 0
    for i in range(1, len(pat)):
        print(f"i: {i}, j: {j}, lps: {lps}", pat[i], pat[j])
        while j > 0 and pat[i] != pat[j]:
            print(f"Mismatch at i: {i}, j: {j}, lps: {lps}")
            j = lps[j-1]  # thu ngắn border
        if pat[i] == pat[j]:
            print(f"Match at i: {i}, j: {j},  {pat[0:i+1]}")
            j += 1
        lps[i] = j
    return lps

print(build_lps("ababaca"))  # Output: [0, 0, 1, 2, 0]
def kmp_search(text, pattern):
    lps = build_lps(pattern)
    i = j = 0
    while i < len(text):
        if text[i] == pattern[j]:
            i += 1
            j += 1
        elif j > 0:
            j = lps[j-1]  # nhảy về prefix trùng
        else:
            i += 1
        if j == len(pattern):
            return i - j
    return -1


# print(kmp_search("abbcababcabc", "aa"))  # Output: 2

Building LPS for pattern: ababaca
i: 1, j: 0, lps: [0, 0, 0, 0, 0, 0, 0] b a
i: 2, j: 0, lps: [0, 0, 0, 0, 0, 0, 0] a a
Match at i: 2, j: 0,  aba
i: 3, j: 1, lps: [0, 0, 1, 0, 0, 0, 0] b b
Match at i: 3, j: 1,  abab
i: 4, j: 2, lps: [0, 0, 1, 2, 0, 0, 0] a a
Match at i: 4, j: 2,  ababa
i: 5, j: 3, lps: [0, 0, 1, 2, 3, 0, 0] c b
Mismatch at i: 5, j: 3, lps: [0, 0, 1, 2, 3, 0, 0]
Mismatch at i: 5, j: 1, lps: [0, 0, 1, 2, 3, 0, 0]
i: 6, j: 0, lps: [0, 0, 1, 2, 3, 0, 0] a a
Match at i: 6, j: 0,  ababaca
[0, 0, 1, 2, 3, 0, 1]
